In [66]:
import pandas as pd
import glob

import warnings
warnings.filterwarnings('ignore')

In [67]:
# 過去12ヶ月分のファイルリストを抽出

# get all file list
file_list = glob.glob("data/top100*csv")
file_list.sort()
#file_list = file_list[:-1]
file_list = file_list[-12:]
file_list

['data/top100_traders_20230901_20230930.csv',
 'data/top100_traders_20231001_20231031.csv',
 'data/top100_traders_20231101_20231130.csv',
 'data/top100_traders_20231201_20231231.csv',
 'data/top100_traders_20240101_20240131.csv',
 'data/top100_traders_20240201_20240229.csv',
 'data/top100_traders_20240301_20240331.csv',
 'data/top100_traders_20240401_20240430.csv',
 'data/top100_traders_20240501_20240531.csv',
 'data/top100_traders_20240601_20240630.csv',
 'data/top100_traders_20240701_20240731.csv',
 'data/top100_traders_20240801_20240831.csv']

# 過去12ヶ月分のデータセット

In [68]:
# read files
df_list = []

for i, file_path in enumerate(file_list):
    print(file_path)
    
    # read a excel file
    df_tmp = pd.read_csv(file_path, encoding="shift-jis")
    
    # add period 
    df_tmp["period"] = file_path.split("_")[-2][:6]
    
    # add soource
    df_tmp["source"] = file_path
    
    # append to list
    df_list.append(df_tmp)

data/top100_traders_20230901_20230930.csv
data/top100_traders_20231001_20231031.csv
data/top100_traders_20231101_20231130.csv
data/top100_traders_20231201_20231231.csv
data/top100_traders_20240101_20240131.csv
data/top100_traders_20240201_20240229.csv
data/top100_traders_20240301_20240331.csv
data/top100_traders_20240401_20240430.csv
data/top100_traders_20240501_20240531.csv
data/top100_traders_20240601_20240630.csv
data/top100_traders_20240701_20240731.csv
data/top100_traders_20240801_20240831.csv


In [69]:
# concat all dataframes
dataset_12months = pd.concat(df_list, ignore_index=True)

# remove the row where 'account_id' is NaN
dataset_12months.dropna(subset=["account_id", "percent_change"], inplace=True, how="any")

# convert data type
dataset_12months["percent_change"] = dataset_12months["percent_change"].astype(float)
dataset_12months["so_cnt"] = dataset_12months["so_cnt"].astype(float)
dataset_12months["account_id"] = dataset_12months["account_id"].astype(str)

# preprocess
dataset_12months.dropna(subset=["percent_change"], inplace=True)
dataset_12months["percent_change"] = dataset_12months["percent_change"] - 100
dataset_12months = dataset_12months[dataset_12months["percent_change"]!=0]

In [70]:
dataset_12months.shape

(97160, 46)

In [71]:
dataset_12months.head()

,platform,account_id,group,balance_lastmonth,withdrawal,UPL_lastmonth,balance_2month_ago,deposit,UPL_2month_ago,equity_lastmonth,...,balance_24,balance_25,balance_26,balance_27,balance_28,balance_29,balance_30,balance_31,period,source
6,TY3_MT4,811620004,live-jpy-09,1758276.0,NaN,NaN,1744080.0,NaN,NaN,1758276.0,...,1741674.0,1757854.0,1757242.0,1753496.0,1753496.0,1758276.0,1758276.0,NaN,202309,data/top100_traders_20230901_20230930.csv
7,TY3_MT4,811620015,live-jpy-10,2038971.0,NaN,NaN,2040671.0,NaN,NaN,2038971.0,...,2038971.0,2038971.0,2038971.0,2038971.0,2038971.0,2038971.0,2038971.0,NaN,202309,data/top100_traders_20230901_20230930.csv
8,TY3_MT4,811620068,live-jpy-09,2465326.0,NaN,NaN,2463664.0,NaN,-12090.0,2465326.0,...,2454332.0,2457606.0,2458647.0,2462078.0,2464216.0,2465326.0,2465326.0,NaN,202309,data/top100_traders_20230901_20230930.csv
53,TY3_MT4,811620146,live-jpy-09,2675152.0,NaN,NaN,2716833.0,NaN,-10390.0,2675152.0,...,2675152.0,2675152.0,2675152.0,2675152.0,2675152.0,2675152.0,2675152.0,NaN,202309,data/top100_traders_20230901_20230930.csv
72,TY3_MT4,811620167,live-jpy-09,1525649.0,NaN,NaN,1458382.0,NaN,3850.0,1525649.0,...,1481488.0,1483269.0,1486197.0,1488642.0,1497059.0,1525649.0,1525649.0,NaN,202309,data/top100_traders_20230901_20230930.csv


In [72]:
dataset_12months["period"].unique()

array(['202309', '202310', '202311', '202312', '202401', '202402',
       '202403', '202404', '202405', '202406', '202407', '202408'],
      dtype=object)

In [73]:
dataset_12months.to_csv("dataset_12months_{}.csv".format(dataset_12months["period"].max()), index=False)

# 過去6ヶ月分のデータセット

In [74]:
def define_class(x, accounts_top200, accounts_worst200):
    if x in accounts_top200[:100]:
        return "上位100"
    elif x in accounts_top200[100:]:
        return "上位101~200"
    elif x in accounts_worst200[:100]:
        return "下位100"
    elif x in accounts_worst200[100:]:
        return "下位101~200"
    else:
        return None

In [75]:
df_target_list = []
df_details_list = []

for i in range(-6, 0):
    # define target file
    target_file = file_list[i]
    print(i+6+1, target_file)

    # extract 6months file list
    ref_index = file_list.index(target_file)
    target_file_list = file_list[ref_index-5:ref_index+1]
    
    # copy target data
    df_target = dataset_12months[dataset_12months["source"]==target_file].copy()
    
    # get top200 and worst200 account id list
    accounts_top200 = df_target.sort_values("percent_change", ascending=False).head(200)["account_id"].tolist()
    accounts_worst200 = df_target.sort_values("percent_change", ascending=True).head(200)["account_id"].tolist()

    # define class1: top200 and worst200
    df_target["class1"] = df_target["account_id"].apply(define_class, args=(accounts_top200, accounts_worst200,))

    # get accounts for 6month plus users
    df_agg = dataset_12months[dataset_12months["source"].isin(target_file_list)].groupby("account_id").agg({"percent_change": ["min", "mean", "max", "count"], "period": "max"})
    
    df_6months = df_agg[
        (df_agg[("percent_change", "min")]>0) & 
        (df_agg[("period", "max")]==dataset_12months[dataset_12months["source"].isin(target_file_list)]["period"].max()) & 
        (df_agg[("percent_change", "count")]==6)
    ]
    accounts_6month = list(df_6months.index)

    # define class2: plus 6months in a row
    df_target["class2"] = df_target["account_id"].apply(lambda x: "6ヶ月連続プラス" if x in accounts_6month else None)
    
    # number of transactions
    user_transactions = {}
    d = []
    
    for path in glob.glob("data/*details*{}".format(target_file.split("_")[-1])):
        #print(path)
        
        # read csv file
        df_details_tmp = pd.read_csv(path)
        
        # define period
        df_details_tmp["period"] = df_target["period"].unique()[0]
        
        # define class
        df_details_tmp["class1"] = df_details_tmp["login"].apply(define_class, args=(accounts_top200, accounts_worst200,))
        df_details_tmp["class2"] = df_details_tmp["login"].apply(lambda x: "6ヶ月連続プラス" if x in accounts_6month else None)
        
        # add to df list
        df_details_list.append(df_details_tmp)
        
        # update transactions dict
        user_transactions.update(dict(df_details_tmp.groupby("login").size()))
        d.append(dict(df_details_tmp.groupby("login").size()))
        
    df_target["# of trades"] = df_target["account_id"].apply(lambda x: user_transactions.get(x))
    df_target["class3"] = df_target["# of trades"].fillna(0).apply(lambda x: "10回以上" if x >= 10 else None)
    
    # add df to df_target_list
    df_target_list.append(df_target)
    
    #print()
    

1 data/top100_traders_20240301_20240331.csv
2 data/top100_traders_20240401_20240430.csv
3 data/top100_traders_20240501_20240531.csv
4 data/top100_traders_20240601_20240630.csv
5 data/top100_traders_20240701_20240731.csv
6 data/top100_traders_20240801_20240831.csv


In [76]:
glob.glob("data/*details*{}".format(target_file.split("_")[-1]))

['data/top200_traders_details20240801_20240831.csv',
 'data/6month_traders_details20240801_20240831.csv',
 'data/worst200_traders_details20240801_20240831.csv']

In [77]:
dataset_6months = pd.concat(df_target_list, ignore_index=True)
dataset_6months.head()

,platform,account_id,group,balance_lastmonth,withdrawal,UPL_lastmonth,balance_2month_ago,deposit,UPL_2month_ago,equity_lastmonth,...,balance_28,balance_29,balance_30,balance_31,period,source,class1,class2,# of trades,class3
0,TY3_MT4,811620004,live-jpy-09,2051718.0,NaN,NaN,2038781.0,NaN,NaN,2051718.0,...,2051718.0,2051718.0,2051718.0,2051718.0,202403,data/top100_traders_20240301_20240331.csv,None,None,NaN,None
1,TY3_MT4,811620068,live-jpy-09,2205957.0,NaN,-2281.0,2240342.0,NaN,-16540.0,2203676.0,...,2205957.0,2205957.0,2205957.0,2205957.0,202403,data/top100_traders_20240301_20240331.csv,None,None,NaN,None
2,TY3_MT4,811620124,live-jpy-09,81931.0,NaN,NaN,76098.0,NaN,NaN,81931.0,...,81931.0,81931.0,81931.0,81931.0,202403,data/top100_traders_20240301_20240331.csv,None,None,NaN,None
3,TY3_MT4,811620131,live-jpy-09,105583.0,NaN,-320.0,101543.0,NaN,NaN,105263.0,...,105583.0,105583.0,105583.0,105583.0,202403,data/top100_traders_20240301_20240331.csv,None,None,NaN,None
4,TY3_MT4,811620146,live-jpy-09,3031972.0,NaN,NaN,2989211.0,NaN,NaN,3031972.0,...,3031972.0,3031972.0,3031972.0,3031972.0,202403,data/top100_traders_20240301_20240331.csv,None,6ヶ月連続プラス,7.0,None


In [78]:
dataset_6months_details = pd.concat(df_details_list, ignore_index=True)
dataset_6months_details.head()

,login,ticket,symbol,profit,swaps,sl,tp,open_time,close_time,side,open_price,close_price,period,class1,class2
0,811620995,40385434,GBPJPY.oj5k,-109900.0,3956.0,0.0,190.768,2024-03-06 13:31:09,2024-03-07 10:02:57,buy,190.568,188.370,202403,下位101~200,None
1,811620995,40398546,GBPJPY.oj5k,18000.0,0.0,0.0,189.100,2024-03-07 12:21:41,2024-03-07 16:53:31,buy,188.651,189.101,202403,下位101~200,None
2,811620995,40408338,GBPJPY.oj5k,3760.0,0.0,0.0,188.640,2024-03-08 08:57:30,2024-03-08 11:14:03,sell,189.414,189.320,202403,下位101~200,None
3,811620995,40422034,GBPJPY.oj5k,-25800.0,-1482.0,0.0,186.600,2024-03-11 11:39:11,2024-03-12 05:45:03,sell,188.427,188.943,202403,下位101~200,None
4,811620995,40429943,GBPJPY.oj5k,-38925.0,-6689.0,0.0,186.300,2024-03-12 09:07:16,2024-03-15 17:49:13,sell,189.066,189.931,202403,下位101~200,None


In [79]:
dataset_6months.to_csv("dataset_6months_{}.csv".format(dataset_6months["period"].max()), index=False)

In [80]:
dataset_6months_details.to_csv("dataset_6months_details_{}.csv".format(dataset_6months_details["period"].max()), index=False)

In [81]:
dataset_6months["period"].unique()

array(['202403', '202404', '202405', '202406', '202407', '202408'],
      dtype=object)

In [82]:
dataset_6months_details[dataset_6months_details["period"]=="202404"]["class1"].value_counts()

class1
下位100        15414
下位101~200     8808
上位100         7750
上位101~200     5888
Name: count, dtype: int64

In [83]:
dataset_6months_details

,login,ticket,symbol,profit,swaps,sl,tp,open_time,close_time,side,open_price,close_price,period,class1,class2
0,811620995,40385434,GBPJPY.oj5k,-109900.0000,3956.0,0.00000,190.768,2024-03-06 13:31:09,2024-03-07 10:02:57,buy,190.56800,188.37000,202403,下位101~200,None
1,811620995,40398546,GBPJPY.oj5k,18000.0000,0.0,0.00000,189.100,2024-03-07 12:21:41,2024-03-07 16:53:31,buy,188.65100,189.10100,202403,下位101~200,None
2,811620995,40408338,GBPJPY.oj5k,3760.0000,0.0,0.00000,188.640,2024-03-08 08:57:30,2024-03-08 11:14:03,sell,189.41400,189.32000,202403,下位101~200,None
3,811620995,40422034,GBPJPY.oj5k,-25800.0000,-1482.0,0.00000,186.600,2024-03-11 11:39:11,2024-03-12 05:45:03,sell,188.42700,188.94300,202403,下位101~200,None
4,811620995,40429943,GBPJPY.oj5k,-38925.0000,-6689.0,0.00000,186.300,2024-03-12 09:07:16,2024-03-15 17:49:13,sell,189.06600,189.93100,202403,下位101~200,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444324,001-009-9847237-001,24886,USD/JPY,-139.2000,0.0,144.51600,0.000,2024-08-29 14:40:21,2024-08-29 15:08:23,buy,144.74600,144.51400,202408,下位101~200,None
444325,001-009-9847237-001,24896,CAD/JPY,-69.0000,0.0,107.46300,0.000,2024-08-29 21:26:22,2024-08-29 21:30:00,buy,107.51900,107.45000,202408,下位101~200,None
444326,001-009-9847237-001,24934,EUR/USD,29.4215,0.0,1.10767,0.000,2024-08-29 21:32:26,2024-08-30 00:20:01,sell,1.10796,1.10767,202408,下位101~200,None
444327,001-009-9847237-001,24944,AUD/JPY,-111.0000,0.0,98.45300,0.000,2024-08-30 07:33:35,2024-08-30 08:30:13,buy,98.56200,98.45100,202408,下位101~200,None
